<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/05wk-2
    .ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [13]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-wcPiCEdYML9-6-Xv5RVbso&si=BbNo6mwCHqwOV0FS>}}

# 2. Imports

In [19]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [20]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 자랑

## A. 성능좋음

*Fashion MNIST*

In [47]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

*발악수준으로 설계한 신경망*

In [56]:
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [49]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [50]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(1., device='cuda:0')

In [51]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8520, device='cuda:0')

*대충대충 설계한 합성곱신경망*

In [52]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [53]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [54]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(0.9466, device='cuda:0')

In [55]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8710, device='cuda:0')

## B. 파라메터적음

In [76]:
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
)
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

## C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109> 

# 4. CNN 핵심레이어 

## A. `torch.nn.ReLU`

**(예시1) 연산방법**

## B. `torch.nn.MaxPool2d`

**(예시1) 연산방법, kernel_size 의 의미**

**(예시2) 정사각형이 아닌 커널**

**(예시3) 이미지크기와 딱 맞지않는 커널일경우?**

## C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride=2**

**(예시2) 이동평균 -- kernel_size 와 stride를 확실히 이해**

**(예시3) 2개의 이미지**

**(예시4) 2개의 out_channels**

> 질문: `conv(img) = img @ What` 을 만족하는 What을 찾을 수 있을까? 